# How to open a new shopping mall in Beijing

ibm_data science capstone-week5

This project builds a dataframe of neighborhoods in Beijing by web scraping the data from Wikipedia page and get the geographical coordinates of the neighborhoods through geopy, and get the map data from Foursquare API, then explore and cluster the neighbors.

### Import Libraries

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


In [2]:
! pip install geocoder

### Web Crawler

In [4]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Neighborhoods_in_Beijing").text

In [5]:
# parse data:  html->beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [6]:
# create a list to store neighborhood data
neighborhoodList = []

In [7]:
# append the data
for row in soup.find_all("div", class_="div-col columns column-width")[0].findAll("li"):
    neighborhoodList.append(row.text)

### Store the data into Dataframe

In [8]:
bj_df = pd.DataFrame({"Neighborhood": neighborhoodList})
bj_df.head()

,Neighborhood
0,Qianmen 前门
1,Tian'anmen 天安门
2,Di'anmen 地安门
3,Chongwenmen 崇文门
4,Xuanwumen 宣武门


In [11]:
#check the size
bj_df.shape

(41, 1)

### Delete the Non-english characters

In [19]:
bj_df_ncn=bj_df.Neighborhood.str.replace('[^a-zA-Z0-9 ]', '')

In [20]:
bj_df_ncn

0          Qianmen 
1        Tiananmen 
2          Dianmen 
3      Chongwenmen 
4        Xuanwumen 
5       Fuchengmen 
6         Xizhimen 
7       Deshengmen 
8        Andingmen 
9         Sanlitun 
10      Dongzhimen 
11     Chaoyangmen 
12     Yongdingmen 
13        Zuoanmen 
14        Youanmen 
15      Guangqumen 
16      Guanganmen 
17          Huashi 
18       Xibianmen 
19       Hepingmen 
20       Fuxingmen 
21      Jianguomen 
22      Gongzhufen 
23      Fangzhuang 
24          Guomao 
25        Hepingli 
26        Pinganli 
27      Beixinqiao 
28      Jiaodaokou 
29         Kuanjie 
30        Wangjing 
31      Wangfujing 
32      Dengshikou 
33        Wudaokou 
34           Xidan 
35         Dongdan 
36    Zhongguancun 
37      Panjiayuan 
38     Beijing CBD 
39        Yayuncun 
40       Shifoying 
Name: Neighborhood, dtype: object

### Changed to dataframe

In [21]:
bj_df_ncn = pd.DataFrame(bj_df_ncn, columns=['Neighborhood']) 

In [22]:
bj_df_ncn

,Neighborhood
0,Qianmen
1,Tiananmen
2,Dianmen
3,Chongwenmen
4,Xuanwumen
5,Fuchengmen
6,Xizhimen
7,Deshengmen
8,Andingmen
9,Sanlitun


###  Get the geographical coordinates

In [23]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{},Beijing,China'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [25]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in bj_df_ncn["Neighborhood"].tolist() ]

### Display the coordinates

In [26]:
coords

[[39.892900000000054, 116.40607000000011],
 [39.90444000000008, 116.39139000000011],
 [39.93943000000007, 116.39301000000012],
 [39.89972000000006, 116.41222000000005],
 [39.898330000000044, 116.36806000000001],
 [39.92295000000007, 116.3478],
 [39.93889000000007, 116.35028000000011],
 [39.94126000000006, 116.37929000000008],
 [39.94382000000007, 116.39952000000005],
 [39.936090000000036, 116.44375000000002],
 [39.93596000000008, 116.43027000000006],
 [39.914600000000064, 116.41671000000008],
 [39.87229000000008, 116.10337000000004],
 [39.869290000000035, 116.43320000000006],
 [39.86554000000007, 116.3575800000001],
 [39.89199000000008, 116.43661000000009],
 [39.88806000000005, 116.34194000000002],
 [39.90750000000003, 116.39723000000004],
 [39.89778000000007, 116.35028000000011],
 [39.89996000000008, 116.37435000000005],
 [39.90556000000004, 116.35111000000006],
 [39.914600000000064, 116.41671000000008],
 [39.90750000000003, 116.39723000000004],
 [40.47831000000008, 116.04172000000005

In [27]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [28]:
# merge the coordinates into the original dataframe
bj_df_ncn['Latitude'] = df_coords['Latitude']
bj_df_ncn['Longitude'] = df_coords['Longitude']

In [30]:
#print new table
print(bj_df_ncn.shape)
bj_df_ncn

(41, 3)


,Neighborhood,Latitude,Longitude
0,Qianmen,39.89290,116.40607
1,Tiananmen,39.90444,116.39139
2,Dianmen,39.93943,116.39301
3,Chongwenmen,39.89972,116.41222
4,Xuanwumen,39.89833,116.36806
5,Fuchengmen,39.92295,116.34780
6,Xizhimen,39.93889,116.35028
7,Deshengmen,39.94126,116.37929
8,Andingmen,39.94382,116.39952
9,Sanlitun,39.93609,116.44375


### save the DataFrame as CSV file

In [32]:
bj_df_ncn.to_csv("bj_df_ncn.csv", index=False)

### Create a map of Beijing with neighbors superimposed on top

In [43]:
# get the coordinates of BEIJING
address = 'Beijing'

geolocator = Nominatim(user_agent = "beijing_explorer")

#in case it runs out of time 
location = geolocator.geocode(address,timeout = None)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Beijing {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Beijing 40.190632, 116.412144.


In [44]:
# create map of Toronto using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(bj_df_ncn['Latitude'],bj_df_ncn['Longitude'], bj_df_ncn['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

#### My hardware is not strong enough to support map downloading

In [45]:
# save the map as HTML file
map_kl.save('map_kl.html')

### Use the Foursquare API to explore the neighborhoods

In [54]:
# define Foursquare Credentials and Version
CLIENT_ID = 'QMA2UISQPW0M2OKMQA1Q4NNVQWKMSVNI4KZLUCQNPWP0WQPK' # your Foursquare ID
CLIENT_SECRET = 'EE5124AOJUDC5ZFKDM4N4L0RCMRKHXXFZV1WKATUXLPDO4S0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QMA2UISQPW0M2OKMQA1Q4NNVQWKMSVNI4KZLUCQNPWP0WQPK
CLIENT_SECRET:EE5124AOJUDC5ZFKDM4N4L0RCMRKHXXFZV1WKATUXLPDO4S0


### get the top 100 venues that are within a radius of 2000 meters.

#### Quotas can easily be exceeded

In [55]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(bj_df_ncn['Latitude'], bj_df_ncn['Longitude'], bj_df_ncn['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [56]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(3188, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Qianmen,39.8929,116.40607,New World Beijing Hotel (北京新世界酒店),39.897296,116.405923,Hotel
1,Qianmen,39.8929,116.40607,North Gate: Temple of Heaven (天坛北门),39.886261,116.406326,Park
2,Qianmen,39.8929,116.40607,Tian Tan Kitchen at New World Beijing Hotel,39.897391,116.405876,Restaurant
3,Qianmen,39.8929,116.40607,Hall of Prayer for Good Harvests (祈年殿),39.882131,116.406780,Temple
4,Qianmen,39.8929,116.40607,Bian Yi Fang (便宜坊烤鸭店),39.895075,116.394550,Peking Duck Restaurant


### check how many venues were returned for each neighorhood

In [57]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Andingmen,100,100,100,100,100,100
Beijing CBD,100,100,100,100,100,100
Chaoyangmen,100,100,100,100,100,100
Chongwenmen,100,100,100,100,100,100
Dengshikou,100,100,100,100,100,100
Deshengmen,100,100,100,100,100,100
Dianmen,100,100,100,100,100,100
Dongdan,100,100,100,100,100,100
Dongzhimen,100,100,100,100,100,100


In [58]:
#Find out how many unique categories can be curated from all the returned venues

print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 176 uniques categories.


In [59]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Hotel', 'Park', 'Restaurant', 'Temple', 'Peking Duck Restaurant',
       'Department Store', 'Cocktail Bar', 'Lounge', 'Yunnan Restaurant',
       'Coffee Shop', 'Beijing Restaurant', 'Chinese Restaurant',
       'Bookstore', 'Japanese Restaurant', 'Performing Arts Venue',
       'Hostel', 'Historic Site', 'Tea Room', 'Café', 'Plaza',
       'Pizza Place', 'Vegetarian / Vegan Restaurant', 'Clothing Store',
       'Shopping Mall', 'History Museum', 'New American Restaurant',
       'Building', 'Theater', 'Hotel Bar', 'Pharmacy',
       'Monument / Landmark', 'Flea Market', 'Fast Food Restaurant',
       'Scenic Lookout', 'Museum', 'Sandwich Place',
       'Street Food Gathering', 'Dim Sum Restaurant', 'Bar',
       'Concert Hall', 'Dumpling Restaurant', 'Sculpture Garden',
       'Zhejiang Restaurant', 'Church', 'Garden', 'Electronics Store',
       'Toy / Game Store', 'Shopping Plaza', 'American Restaurant',
       'Modern European Restaurant'], dtype=object)

In [61]:
# check if the results contain "Shopping Mall"
"Shopping Mall" in venues_df['VenueCategory'].unique()

True

### Analyze Each Neighborhood

In [62]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(3188, 177)


,Neighborhoods,African Restaurant,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bar,Beer Bar,Beer Garden,Beijing Restaurant,Bistro,Bookstore,Brewery,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Burger Joint,Butcher,Café,Cantonese Restaurant,Chinese Aristocrat Restaurant,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Concert Hall,Convenience Store,Coworking Space,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Exhibit,Fabric Shop,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Garden,German Restaurant,Gourmet Shop,Grocery Store,Gym / Fitness Center,Halal Restaurant,Herbs & Spices Store,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Hubei Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jiangsu Restaurant,Jiangxi Restaurant,Karaoke Bar,Korean Restaurant,Lake,Lounge,Malay Restaurant,Massage Studio,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Opera House,Optical Shop,Outdoor Sculpture,Park,Pedestrian Plaza,Peking Duck Restaurant,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pharmacy,Pizza Place,Planetarium,Platform,Plaza,Pub,Public Art,Ramen Restaurant,Resort,Restaurant,Road,Rock Climbing Spot,Rock Club,Russian Restaurant,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shaanxi Restaurant,Shandong Restaurant,Shanxi Restaurant,Shopping Mall,Shopping Plaza,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Taxi Stand,Tea Room,Temple,Thai Restaurant,Theater,Theme Park,Tibetan Restaurant,Tiki Bar,Toy / Game Store,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint,Xinjiang Restaurant,Yoga Studio,Yunnan Restaurant,Zhejiang Restaurant,Zoo Exhibit
0,Qianmen,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Qianmen,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Qianmen,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Qianmen,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [63]:
#group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(38, 177)


,Neighborhoods,African Restaurant,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bar,Beer Bar,Beer Garden,Beijing Restaurant,Bistro,Bookstore,Brewery,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Burger Joint,Butcher,Café,Cantonese Restaurant,Chinese Aristocrat Restaurant,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Concert Hall,Convenience Store,Coworking Space,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Exhibit,Fabric Shop,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Garden,German Restaurant,Gourmet Shop,Grocery Store,Gym / Fitness Center,Halal Restaurant,Herbs & Spices Store,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Hubei Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jiangsu Restaurant,Jiangxi Restaurant,Karaoke Bar,Korean Restaurant,Lake,Lounge,Malay Restaurant,Massage Studio,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Opera House,Optical Shop,Outdoor Sculpture,Park,Pedestrian Plaza,Peking Duck Restaurant,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pharmacy,Pizza Place,Planetarium,Platform,Plaza,Pub,Public Art,Ramen Restaurant,Resort,Restaurant,Road,Rock Climbing Spot,Rock Club,Russian Restaurant,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shaanxi Restaurant,Shandong Restaurant,Shanxi Restaurant,Shopping Mall,Shopping Plaza,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Taxi Stand,Tea Room,Temple,Thai Restaurant,Theater,Theme Park,Tibetan Restaurant,Tiki Bar,Toy / Game Store,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint,Xinjiang Restaurant,Yoga Studio,Yunnan Restaurant,Zhejiang Restaurant,Zoo Exhibit
0,Andingmen,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.010000,0.020000,0.000000,0.100000,0.000000,0.010000,0.000000,0.01,0.000000,0.01,0.000000,0.01,0.00,0.00,0.010000,0.01,0.090000,0.000000,0.00,0.130000,0.00,0.00,0.000000,0.02,0.040000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.010000,0.000000,0.000000,0.020000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.02,0.01,0.010000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.040000,0.000000,0.00,0.040000,0.020000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.00,0.030000,0.01,0.000000,0.000000,0.00,0.00,0.000000,0.01000,0.010000,0.010000,0.00,0.000000,0.010000,0.000000,0.01,0.00000,0.00,0.020000,0.00,0.000000,0.000000,0.000000,0.00,0.010000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.010000,0.00,0.010000,0.000000,0.00,0.01,0.000000,0.010000,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,0.020000,0.00,0.000000,0.01,0.000000,0.000000,0.00,0.000000,0.00000,0.00,0.00,0.000000,0.00,0.01,0.00,0.00,0.000000,0.01,0.00,0.000000,0.00,0.00,0.000000,0.01,0.010000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.010000,0.00000,0.00,0.01,0.00,0.000000,0.00,0.030000,0.01,0.00,0.00,0.000000,0.00,0.030000,0.00,0.000000
1,Beijing CBD,0.00,0.01,0.000000,0.000000,0.00,0.000000,0.010000,

In [64]:
len(kl_grouped[kl_grouped["Shopping Mall"] > 0])

31

### Create shopping mall dataframe

In [65]:
kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]

In [66]:
kl_mall.head()

,Neighborhoods,Shopping Mall
0,Andingmen,0.00
1,Beijing CBD,0.02
2,Chaoyangmen,0.03
3,Chongwenmen,0.02
4,Dengshikou,0.02


### Cluster Neighborhoods

In [67]:
#Run k-means to cluster the neighborhoods in Kuala Lumpur into 3 clusters.
# set number of clusters
kclusters = 3
 
kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 0, 0, 0, 1, 1, 0, 0, 2])

In [68]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [69]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Andingmen,0.00,1
1,Beijing CBD,0.02,0
2,Chaoyangmen,0.03,0
3,Chongwenmen,0.02,0
4,Dengshikou,0.02,0


In [70]:
# add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(bj_df_ncn.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(38, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Andingmen,0.00,1,39.94382,116.39952
1,Beijing CBD,0.02,0,39.90750,116.39723
2,Chaoyangmen,0.03,0,39.91460,116.41671
3,Chongwenmen,0.02,0,39.89972,116.41222
4,Dengshikou,0.02,0,39.90750,116.39723


In [71]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(38, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
18,Jianguomen,0.030000,0,39.91460,116.41671
33,Yayuncun,0.021739,0,40.01388,116.39644
31,Xizhimen,0.016393,0,39.93889,116.35028
30,Xidan,0.020000,0,39.90750,116.39723
27,Wangjing,0.030000,0,39.99330,116.47284
25,Shifoying,0.020000,0,39.90750,116.39723
24,Sanlitun,0.020000,0,39.93609,116.44375
22,Pinganli,0.020000,0,39.90750,116.39723
20,Kuanjie,0.020000,0,39.90750,116.39723
36,Zhongguancun,0.030000,0,39.98111,116.30889


### visualize cluster

In [72]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [73]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### Examine Clusters

#### Cluster 1

In [74]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
18,Jianguomen,0.030000,0,39.91460,116.41671
33,Yayuncun,0.021739,0,40.01388,116.39644
31,Xizhimen,0.016393,0,39.93889,116.35028
30,Xidan,0.020000,0,39.90750,116.39723
27,Wangjing,0.030000,0,39.99330,116.47284
25,Shifoying,0.020000,0,39.90750,116.39723
24,Sanlitun,0.020000,0,39.93609,116.44375
22,Pinganli,0.020000,0,39.90750,116.39723
20,Kuanjie,0.020000,0,39.90750,116.39723
36,Zhongguancun,0.030000,0,39.98111,116.30889


#### Cluster 2

In [75]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
12,Guanganmen,0.00,1,39.88806,116.34194
34,Yongdingmen,0.00,1,39.87229,116.10337
28,Wudaokou,0.00,1,39.99257,116.33208
13,Guangqumen,0.01,1,39.89199,116.43661
5,Deshengmen,0.00,1,39.94126,116.37929
19,Jiaodaokou,0.01,1,39.93987,116.40677
15,Hepingli,0.00,1,39.95276,116.41093
6,Dianmen,0.00,1,39.93943,116.39301
0,Andingmen,0.00,1,39.94382,116.39952


#### Cluster 3

In [76]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
26,Tiananmen,0.040000,2,39.90444,116.39139
23,Qianmen,0.040000,2,39.89290,116.40607
21,Panjiayuan,0.057143,2,39.88392,116.47576
29,Xibianmen,0.053333,2,39.89778,116.35028
9,Fuchengmen,0.049180,2,39.92295,116.34780
32,Xuanwumen,0.040541,2,39.89833,116.36806
10,Fuxingmen,0.043478,2,39.90556,116.35111
35,Youanmen,0.060606,2,39.86554,116.35758


### Conclusion

Cluster 1 has more communities than Cluster 2 and 3，but has moderate number of shopping centers，Cluster 3 has the most shopping centers，
the competition in this area may be fierce and the supply exceeds the demand. There are almost no shopping centers near the communities in
cluster 2, so cluster 2 has great potential to open new shopping centers. Therefore, it is advocated that developers mainly open new 
shopping centers near Cluster 2